In [1]:
from dotenv import load_dotenv
import requests
import json
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# plt.style.use('classic')
%matplotlib inline

import seaborn as sns
# sns.set()
# plt.rcParams['figure.figsize'] = (31,7)

In [2]:
cacheDir = '~/pandas_cache'

load_dotenv()
HASURA_SECRET = os.getenv("HASURA_SECRET")

In [3]:
def run_query(query, limit=1000000, offset=None):
    
    if offset is None:
        return_df = None
        offset = 0;
        while True:
            df = run_query(query, limit, offset)
            if return_df is None:
                return_df = df.copy()
            else:
                return_df = pd.concat([return_df, df], ignore_index=True)
            print(f"{len(return_df)} rows")
            if len(df) < limit:
                break
            else:
                offset += limit
        return return_df        
    
    url = 'http://localhost:8080/v1/query'
    headers = {"content-type": "application/json", 
               "x-hasura-admin-secret": HASURA_SECRET
              }
    try:
        body = {
            "type": "run_sql",
            "args": {
                "sql": f"{query} limit {limit} offset {offset}"
            }
        }
        r = requests.post(url, json=body, headers=headers)
        results = json.loads(r.text)["result"]
        return pd.DataFrame(results[1:], columns=results[0])
    except Exception as error:
        print(error)
        print(r.status_code)
        print(r.text)

In [4]:
if not os.path.exists(cacheDir):
  os.makedirs(cacheDir)

def cache_query(name, query, munger=None, use_cache=True):
    fname = f"~/pandas_cache/{name}.pkl"
    try:
        if not use_cache:
            raise Exception("no cache")
        df = pd.read_pickle(fname)
        print("read from cache")
        print(f"{len(df)} rows")
        return df
    except:
        print("running database query")
        df = run_query(query)
        df.to_pickle(fname)
        if munger:
            df = munger(df)
        df.to_pickle(fname)
        return df 

In [5]:
def query_year(year):
    return f"""
select 
    n.created_at, n.sent_at, n.updated_at, n.notification_status as status,
    extract(epoch from (n.sent_at - n.created_at)) as processing_seconds,
    extract(epoch from (n.updated_at - n.sent_at)) as callback_seconds,
    t.process_type as priority,
    s.count_as_live,
    s.name
from 
    notification_history n
    join services s on n.service_id = s.id
    join templates_history t on n.template_id = t.id and n.template_version = t.version
where
    s.count_as_live and
    extract(YEAR from n.created_at) = {year}
order by created_at
"""

In [6]:
def munge_df(df):
    print("munging data")
    df = df.replace("NULL", np.nan)
    df['created_at'] = pd.to_datetime(df.created_at)
    df['updated_at'] = pd.to_datetime(df.updated_at)
    df['date'] = df.created_at.dt.date
    df['month'] = df.created_at.dt.to_period('M')
    df['processing_seconds'] = pd.to_numeric(df.processing_seconds,  errors='coerce')
    df['callback_seconds'] = pd.to_numeric(df.callback_seconds,  errors='coerce')
    return df

## Get data

query / munging times:
* 2020: 7min 5s
* 2021: 30 min?
* 2022: 30 min?

In [7]:
%time df = cache_query('notification_history_2022', query_year(2022), munger=munge_df)

read from cache
16824277 rows
CPU times: user 16.1 s, sys: 12.3 s, total: 28.5 s
Wall time: 32.1 s


## Monthly amount of notifications

In [8]:
month_counts = df[['month']].value_counts()
pd.DataFrame(month_counts).sort_values(by='month')

,0
month,
2022-01,2300506
2022-02,2410295
2022-03,4011291
2022-04,4930607
2022-05,3165550
2022-06,6028


## By priority

In [9]:
df_counts = df[['month', 'priority', 'processing_seconds']].groupby(['month', 'priority']).count()
df_counts.columns = ['count']
df_counts = df_counts.reset_index()
df_counts['percent'] = df_counts.apply(lambda r: r['count'] / month_counts[r['month']] * 100, axis=1)
df_counts.set_index(['month', 'priority'])

count     percent
month   priority                     
2022-01 bulk       424493   18.452158
        normal    1859685   80.838085
        priority    16326    0.709670
2022-02 bulk       523494   21.719084
        normal    1862528   77.273861
        priority    24267    1.006806
2022-03 bulk      1695289   42.262927
        normal    2269286   56.572460
        priority    46707    1.164388
2022-04 bulk      2406941   48.816322
        normal    2456123   49.813806
        priority    67508    1.369162
2022-05 bulk       480625   15.182986
        normal    2302800   72.745652
        priority   382082   12.070004
2022-06 normal       6028  100.000000

## Quantiles

In [10]:
df_p = df[['month', 'priority', 'processing_seconds']]
df_50 = df_p.groupby(['month', 'priority']).quantile(q=0.5)
df_50.columns = ['processing_50']
df_90 = df_p.groupby(['month', 'priority']).quantile(q=0.9)
df_90.columns = ['processing_90']
df_99 = df_p.groupby(['month', 'priority']).quantile(q=0.99)
df_99.columns = ['processing_99']
processing_df = pd.concat([df_50, df_90, df_99], axis=1).reset_index()
processing_df.set_index(['month', 'priority'])

processing_50  processing_90  processing_99
month   priority                                             
2022-01 bulk          19.673351      48.002122      67.968990
        normal         6.735093      33.786717      77.537271
        priority       0.166463       0.694634       4.043634
2022-02 bulk          17.417766     880.084007    2614.830909
        normal         5.811389      34.615675      78.181409
        priority       0.169047       1.298963       4.609431
2022-03 bulk         896.016130    2187.190259    2934.161517
        normal         3.321929     953.887996    3156.208244
        priority       0.188487       0.931116       3.711454
2022-04 bulk        1237.207210    2607.158176    3373.323757
        normal         2.290278     505.948825    1178.791027
        priority       0.210094       1.334301       4.522519
2022-05 bulk         349.244555    1509.730457    2350.173067
        normal        11.165273     965.449814    1676.602978
        priority       1.171855       7.565883      19.304561
2022-06 normal         6.074083     251.711924     576.721906

## Busiest hours

For each month, find the busiest hour.
A burst can be split across 2 hours, so when doing perf tests, to be safe we should double the size of the largest hour.

In [11]:
df['hour'] = df.created_at.dt.round('H')
df2 = pd.DataFrame(df[['month', 'hour']].value_counts()).reset_index()
df2.columns = ['month', 'hour', 'count']
df2 = df2.sort_values(by=['month', 'count']).reset_index(drop=True)
df2.groupby('month').last()

,hour,count
month,,
2022-01,2022-01-18 19:00:00,62250
2022-02,2022-02-09 22:00:00,100786
2022-03,2022-03-29 18:00:00,109451
2022-04,2022-04-19 13:00:00,124671
2022-05,2022-05-09 15:00:00,74281
2022-06,2022-06-03 02:00:00,575


In [12]:
# check
df2[['month', 'count']].groupby('month').max()

,count
month,
2022-01,62250
2022-02,100786
2022-03,109451
2022-04,124671
2022-05,74281
2022-06,575
